<center><h1>Wang_Jason_HW7 </h1></center>

Name: Jason Wang
<br>
Github Username: jasonwangg1321 
<br>
USC ID: 5363148736

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

Import packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import hamming_loss





### (a) Download the Anuran Calls (MFCCs) Data Set

In [2]:
df = pd.read_csv('Frogs_MFCCs.csv')
display(df.head())

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [3]:
training, testing = train_test_split(df, train_size= 0.7, random_state=42)

### (b) Train a classifier for each label

#### (i) Research

From https://mmuratarat.github.io/2020-01-25/multilabel_classification_metrics,

Exact Match Ratio

One trivial way around would just to ignore partially correct (consider them incorrect) and extend the accuracy used in single label case for multi-label prediction.

Exact Match Ratio, MR=1n∑i=1nI(yi=yi^)

where I
 is the indicator function. Clearly, a disadvantage of this measure is that it does not distinguish between complete incorrect and partially correct which might be considered harsh.

Hamming Loss
It reports how many times on average, the relevance of an example to a class label is incorrectly predicted. Therefore, hamming loss takes into account the prediction error (an incorrect label is predicted) and missing error (a relevant label not predicted), normalized over total number of classes and total number of examples.

Hamming Loss=1nL∑i=1n∑j=1LI(yji≠y^ji)

where I
 is the indicator function. Ideally, we would expect the hamming loss to be 0, which would imply no error; practically the smaller the value of hamming loss, the better the performance of the learning algorithm.

#### (ii) Train a SVM for each of the labels

In [4]:
# https://scikit-learn.org/stable/modules/svm.html 

# train svm for each of the labels using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernal using 10 fold cross validation.

x_training = training.iloc[0:,0:-4]
y_training = training.iloc[0:,-4:-1] 
family_training_label = y_training['Family']
genus_training_label = y_training['Genus']
species_training_label = y_training['Species']

x_testing = testing.iloc[0:,0:-4]
y_testing = testing.iloc[0:,-4:-1] 
family_testing_label = y_testing['Family']
genus_testing_label = y_testing['Genus']
species_testing_label = y_testing['Species']

In [5]:
#https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf used this guide to determine C and gamma parameters to conduct loose and fine grid search (page 5)

#Loose Grid Search 
estimator = svm.SVC(kernel='rbf') # ovr is default 
param_grid = {'C': [2**i for i in range (-5,12,2)],
              'gamma': [2**i for i in range(-11,4,2)]}
grid_search = GridSearchCV(estimator, param_grid, cv = 10, scoring= 'accuracy')

family = grid_search.fit(x_training, family_training_label)
print(family.best_params_)
print(f"family accuracy: {family.best_score_}")

genus = grid_search.fit(x_training, genus_training_label)
print(genus.best_params_)
print(f"genus accuracy: {genus.best_score_}")

species = grid_search.fit(x_training, species_training_label)
print(species.best_params_)
print(f"species accuracy: {species.best_score_}")

{'C': 8, 'gamma': 2}
family accuracy: 0.9932492347502289
{'C': 32, 'gamma': 2}
genus accuracy: 0.9902687052289437
{'C': 8, 'gamma': 2}
species accuracy: 0.9904675123860013


In [6]:
# fine grid search, tune in on the parameters from the above. can see C values range from 2 to 128 while gamma is consistent at 2, changing step to 1 to tune in. 
# Note that on different runs, i got different values of C due to shuffling, so the parameter values I used may be different from the above but did not want to change due to run times.
estimator = svm.SVC(kernel='rbf')
fine_param_grid = {'C': [2**i for i in range (1,7,1)],
              'gamma': [2**i for i in range(-1,3,1)]}
fine_grid_search = GridSearchCV(estimator, fine_param_grid, cv = 10, scoring= 'accuracy')
family2 = fine_grid_search.fit(x_training, family_training_label)
print(family2.best_params_)
print(f"family accuracy: {family2.best_score_}")

genus2 = fine_grid_search.fit(x_training, genus_training_label)
print(genus2.best_params_)
print(f"genus accuracy: {genus2.best_score_}")

species2 = fine_grid_search.fit(x_training, species_training_label)
print(species2.best_params_)
print(f"species accuracy: {species2.best_score_}")

{'C': 16, 'gamma': 2}
family accuracy: 0.9936464546056991
{'C': 16, 'gamma': 2}
genus accuracy: 0.9904675123860013
{'C': 16, 'gamma': 2}
species accuracy: 0.990665925084414


On some runs, I didnt see any improvements in accuracy from fine tuning the grid search because the accuracy was already high to begin with, but we different C and Gamma values (on previous run). In this particular run, I saw improvements in accuracy.

In [7]:
# select fixed parameters (20) between these the above points in log
final_param_grid = {'C': np.logspace(1,7,20,base=2),
              'gamma': np.linspace(1,2,20)}
final_grid_search = GridSearchCV(estimator, final_param_grid, cv = 10, scoring= 'accuracy')

final_family = final_grid_search.fit(x_training, family_training_label)
best_family = final_family.best_estimator_
best_family = final_family.predict(x_testing)

family_hamming_loss = hamming_loss(family_testing_label, best_family)
family_hamming_score = 1 - family_hamming_loss
family_pred_score = accuracy_score(family_testing_label,best_family)

print('Family')
print(final_family.best_params_)
print(f"accuracy: {final_family.best_score_}")
print(f'Test Accuracy: {family_pred_score}')
print(f'Test Hamming Loss: {family_hamming_loss}')
print(f'Test Hamming Score: {family_hamming_score}')

final_genus = final_grid_search.fit(x_training, genus_training_label)
best_genus = final_genus.best_estimator_
best_genus = final_genus.predict(x_testing)

genus_hamming_loss = hamming_loss(genus_testing_label, best_genus)
genus_hamming_score = 1 - genus_hamming_loss
genus_pred_score = accuracy_score(genus_testing_label,best_genus)

print('Genus')
print(final_genus.best_params_)
print(f"accuracy: {final_genus.best_score_}")
print(f'Test Accuracy: {genus_pred_score}' )
print(f'Test Hamming Loss: {genus_hamming_loss}')
print(f'Test Hamming Score: {genus_hamming_score}')

final_species = final_grid_search.fit(x_training, species_training_label)
best_species = final_species.best_estimator_
best_species = final_species.predict(x_testing)

species_hamming_loss = hamming_loss(species_testing_label, best_species)
species_hamming_score = 1 - species_hamming_loss
species_pred_score = accuracy_score(species_testing_label,best_species)

print('Species')
print(final_species.best_params_)
print(f"accuracy: {final_species.best_score_}")
print(f'Test Accuracy: {species_pred_score}' )
print(f'Test Hamming Loss: {species_hamming_loss}')
print(f'Test Hamming Score: {species_hamming_score}')


Family
{'C': 17.8505266841847, 'gamma': 2.0}
accuracy: 0.9936464546056991
Test Accuracy: 0.9949050486336267
Test Hamming Loss: 0.005094951366373321
Test Hamming Score: 0.9949050486336267
Genus
{'C': 17.8505266841847, 'gamma': 1.526315789473684}
accuracy: 0.9908647322414719
Test Accuracy: 0.9898100972672533
Test Hamming Loss: 0.010189902732746642
Test Hamming Score: 0.9898100972672533
Species
{'C': 9.256881898261756, 'gamma': 1.894736842105263}
accuracy: 0.990666319543059
Test Accuracy: 0.9902732746641963
Test Hamming Loss: 0.009726725335803613
Test Hamming Score: 0.9902732746641963


#### (iii) Repeat 1(b)ii with L1-penalized SVMs

In [8]:
#Loose Grid Search 
pipeline = Pipeline([('scaler', StandardScaler()),
                    ('svc', svm.LinearSVC(penalty = 'l1', dual=False, max_iter= 100000)) ## used chatgpt with pipeline portion, was encountering errors with C values and convergence when attempting l1
                    ])

param_grid = {'svc__C': [2**i for i in range (-5,12,2)]}
grid_search_l1 = GridSearchCV(pipeline, param_grid, cv = 10, scoring= 'accuracy')

family_l1 = grid_search_l1.fit(x_training, family_training_label)
print(family_l1.best_params_)
print(f"family accuracy: {family_l1.best_score_}")

genus_l1 = grid_search_l1.fit(x_training, genus_training_label)
print(genus_l1.best_params_)
print(f"genus accuracy: {genus_l1.best_score_}")

species_l1 = grid_search_l1.fit(x_training, species_training_label)
print(species_l1.best_params_)
print(f"species accuracy: {species_l1.best_score_}")

{'svc__C': 0.5}
family accuracy: 0.9408280475874911
{'svc__C': 8}
genus accuracy: 0.9525434693426741
{'svc__C': 2}
species accuracy: 0.960285114708574


In [9]:
#fine grid search, we can see C falls between ~0.5 to 8
param_grid = {'svc__C': [2**i for i in range (-1,3,1)]}
fine_grid_search_l1 = GridSearchCV(pipeline, param_grid, cv = 10, scoring= 'accuracy')

family_l1_2 = fine_grid_search_l1.fit(x_training, family_training_label)
print(family_l1_2.best_params_)
print(f"family accuracy: {family_l1_2.best_score_}")

genus_l1_2 = fine_grid_search_l1.fit(x_training, genus_training_label)
print(genus_l1_2.best_params_)
print(f"genus accuracy: {genus_l1_2.best_score_}")

species_l1_2 = fine_grid_search_l1.fit(x_training, species_training_label)
print(species_l1_2.best_params_)
print(f"species accuracy: {species_l1_2.best_score_}")

{'svc__C': 0.5}
family accuracy: 0.9408280475874911
{'svc__C': 4}
genus accuracy: 0.9525430748840291
{'svc__C': 2}
species accuracy: 0.960285114708574


In [10]:
# we see from the fine grd search, C values are from 0.5 to 4 so we can adjust our final param grid accordingly

final_l1_param_grid = {'C': np.logspace(-1,2,20,base=2)}
final_l1_grid_search = GridSearchCV(estimator, final_l1_param_grid, cv = 10, scoring= 'accuracy')
final_family_l1 = final_l1_grid_search.fit(x_training, family_training_label)

best_family_l1 = final_family_l1.best_estimator_
best_family_l1 = final_family_l1.predict(x_testing)

family_l1_hamming_loss = hamming_loss(family_testing_label, best_family_l1)
family_l1_hamming_score = 1 - family_l1_hamming_loss
family_l1_pred_score = accuracy_score(family_testing_label,best_family_l1)

print('Family')
print(final_family_l1.best_params_)
print(f"Family accuracy: {final_family_l1.best_score_}")
print(f'Family Test Accuracy: {family_l1_pred_score}' )
print(f'Test Hamming Loss: {family_l1_hamming_loss}')
print(f'Test Hamming Score: {family_l1_hamming_score}')

final_genus_l1 = final_l1_grid_search.fit(x_training, genus_training_label)
best_genus_l1 = final_genus_l1.best_estimator_
best_genus_l1 = final_genus_l1.predict(x_testing)

genus_l1_hamming_loss = hamming_loss(genus_testing_label, best_genus_l1)
genus_l1_hamming_score = 1 - genus_l1_hamming_loss
genus_l1_pred_score = accuracy_score(genus_testing_label,best_genus_l1)

print('Genus')
print(final_genus_l1.best_params_)
print(f"Genus accuracy: {final_genus_l1.best_score_}")
print(f'Genus Test Accuracy: {genus_l1_pred_score}' )
print(f'Test Hamming Loss: {genus_l1_hamming_loss}')
print(f'Test Hamming Score: {genus_l1_hamming_score}')


final_species_l1 = final_l1_grid_search.fit(x_training, species_training_label)
best_species_l1 = final_species_l1.best_estimator_
best_species_l1 = final_species_l1.predict(x_testing)

species_l1_hamming_loss = hamming_loss(species_testing_label, best_species_l1)
species_l1_hamming_score = 1 - genus_l1_hamming_loss
species_l1_pred_score = accuracy_score(species_testing_label,best_species_l1)

print('Species')
print(final_species_l1.best_params_)
print(f"Species accuracy: {final_species_l1.best_score_}")
print(f'Species Test Accuracy: {species_l1_pred_score}' )
print(f'Test Hamming Loss: {species_l1_hamming_loss}')
print(f'Test Hamming Score: {species_l1_hamming_score}')


Family
{'C': 4.0}
Family accuracy: 0.9868952948972829
Family Test Accuracy: 0.984251968503937
Test Hamming Loss: 0.015748031496062992
Test Hamming Score: 0.984251968503937
Genus
{'C': 4.0}
Genus accuracy: 0.9872917258354637
Genus Test Accuracy: 0.9851783232978231
Test Hamming Loss: 0.014821676702176934
Test Hamming Score: 0.9851783232978231
Species
{'C': 4.0}
Species accuracy: 0.9870921297611158
Species Test Accuracy: 0.9865678554886521
Test Hamming Loss: 0.013432144511347846
Test Hamming Score: 0.9851783232978231


#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [11]:
smote = SMOTE(sampling_strategy='auto', random_state = 60)
x_training_resampled1, family_training_resampled  = smote.fit_resample(x_training, family_training_label)
x_training_resampled2, genus_training_resampled  = smote.fit_resample(x_training, genus_training_label)
x_training_resampled3, species_training_resampled  = smote.fit_resample(x_training, species_training_label)


In [12]:
pipeline = Pipeline([('scaler', StandardScaler()),
                    ('svc', svm.LinearSVC(penalty = 'l1', dual=False, max_iter= 100000))
                    ])

param_grid = {'svc__C': [2**i for i in range (-5,12,2)]}
grid_search_smote = GridSearchCV(pipeline, param_grid, cv = 10, scoring= 'accuracy')

family_smote = grid_search_smote.fit(x_training_resampled1, family_training_resampled)
print(family_smote.best_params_)
print(f"family accuracy: {family_smote.best_score_}")

genus_smote = grid_search_smote.fit(x_training_resampled2, genus_training_resampled)
print(genus_smote.best_params_)
print(f"genus accuracy: {genus_smote.best_score_}")

species_smote = grid_search_smote.fit(x_training_resampled3, species_training_resampled)
print(species_smote.best_params_)
print(f"species accuracy: {species_smote.best_score_}")


{'svc__C': 128}
family accuracy: 0.9509448490742027
{'svc__C': 8}
genus accuracy: 0.9542090638057428
{'svc__C': 8}
species accuracy: 0.9615038823048631


In [13]:
#fine grid search, we can see C falls between ~8 to 128
param_grid = {'svc__C': [2**i for i in range (3,8,1)]}
grid_search_smote = GridSearchCV(pipeline, param_grid, cv = 10, scoring= 'accuracy')

family_smote_2 = grid_search_smote.fit(x_training_resampled1, family_training_resampled)
print(family_smote_2.best_params_)
print(f"family accuracy: {family_smote_2.best_score_}")

genus_smote_2 = fine_grid_search_l1.fit(x_training_resampled2, genus_training_resampled)
print(genus_smote_2.best_params_)
print(f"genus accuracy: {genus_smote_2.best_score_}")

species_smote_2 = fine_grid_search_l1.fit(x_training_resampled3, species_training_resampled)
print(species_smote_2.best_params_)
print(f"species accuracy: {species_smote_2.best_score_}")

{'svc__C': 64}
family accuracy: 0.9509448490742027
{'svc__C': 4}
genus accuracy: 0.9541228197350227
{'svc__C': 4}
species accuracy: 0.961463015937883


In [14]:
# we see from the fine grid search, C values are from 4 to 64 so we can adjust our final param grid accordingly

final_smote_param_grid = {'C': np.logspace(2,6,20,base=2)}
final_smote_grid_search = GridSearchCV(estimator, final_l1_param_grid, cv = 10, scoring= 'accuracy')
final_family_smote = final_smote_grid_search.fit(x_training_resampled1, family_training_resampled)

best_family_smote = final_family_smote.best_estimator_
best_family_smote = final_family_smote.predict(x_testing)


family_smote_hamming_loss = hamming_loss(family_testing_label, best_family_smote)
family_smote_hamming_score = 1 - family_smote_hamming_loss
family_smote_pred_score = accuracy_score(family_testing_label,best_family_smote)

print('Family')
print(final_family_smote.best_params_)
print(f"Family accuracy: {final_family_smote.best_score_}")
print(f'Family Test Accuracy: {family_smote_pred_score}' )
print(f'Test Hamming Loss: {family_smote_hamming_loss}')
print(f'Test Hamming Score: {family_smote_hamming_score}')

final_genus_smote = final_smote_grid_search.fit(x_training_resampled2, genus_training_resampled)
best_genus_smote = final_genus_smote.best_estimator_
best_genus_smote = final_genus_smote.predict(x_testing)

genus_smote_hamming_loss = hamming_loss(genus_testing_label, best_genus_smote)
genus_smote_hamming_score = 1 - genus_smote_hamming_loss
genus_smote_pred_score = accuracy_score(genus_testing_label,best_genus_smote)

print('Genus')
print(final_genus_smote.best_params_)
print(f"Genus accuracy: {final_genus_smote.best_score_}")
print(f'Genus Test Accuracy: {genus_smote_pred_score}' )
print(f'Test Hamming Loss: {genus_smote_hamming_loss}')
print(f'Test Hamming Score: {genus_smote_hamming_score}')


final_species_smote = final_smote_grid_search.fit(x_training_resampled3, species_training_resampled)
best_species_smote = final_species_smote.best_estimator_
best_species_smote = final_species_smote.predict(x_testing)

species_smote_hamming_loss = hamming_loss(species_testing_label, best_species_smote)
species_smote_hamming_score = 1 - species_smote_hamming_loss
species_smote_pred_score = accuracy_score(species_testing_label,best_species_smote)

print('Species')
print(final_species_smote.best_params_)
print(f"Species accuracy: {final_species_smote.best_score_}")
print(f'Species Test Accuracy: {species_smote_pred_score}' )
print(f'Test Hamming Loss: {species_smote_hamming_loss}')
print(f'Test Hamming Score: {species_smote_hamming_score}')



Family
{'C': 3.585328384551423}
Family accuracy: 0.9959323794214345
Family Test Accuracy: 0.9879573876794813
Test Hamming Loss: 0.012042612320518759
Test Hamming Score: 0.9879573876794813
Genus
{'C': 4.0}
Genus accuracy: 0.9982753044564392
Genus Test Accuracy: 0.9884205650764243
Test Hamming Loss: 0.01157943492357573
Test Hamming Score: 0.9884205650764243
Species
{'C': 3.585328384551423}
Species accuracy: 0.9987740089906006
Species Test Accuracy: 0.9879573876794813
Test Hamming Loss: 0.012042612320518759
Test Hamming Score: 0.9879573876794813


We can see that the results from iv (smote) had the best test accuracy out of all of the classifiers

## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) Use k-means clustering

In [15]:
df_features = df.iloc[0:,0:-4]
df_labels = df.iloc[0:,-4:-1]

In [16]:
#https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

silhouette = []
# the below is taken for the scikit-learn website performing silhouette analysis
range_n_clusters = [i for i in range(2,51)]
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters= n_clusters, n_init= 'auto')
    cluster_labels = clusterer.fit_predict(df_features)
    silhouette_avg = silhouette_score(df_features, cluster_labels)
    sample_silhouette_values = silhouette_samples(df_features, cluster_labels)
    silhouette.append(silhouette_avg)


In [17]:
best_k = silhouette.index(max(silhouette)) + 2
best_k


5

### (b) Determine which family is the majority

In [18]:
kmeans = KMeans(n_clusters= best_k, n_init= 'auto')
kmeans.fit(df_features)
labels = kmeans.labels_

In [19]:
df_labels['labels'] = labels
majority = []

# used chatgpt below to help find the majority cluster, was having trouble finding the majority of each label and how to help split into sublists in order to create the majority data frame 
for n in range(best_k):
    cluster_no = df_labels[df_labels['labels'] == n]
    family_majority = cluster_no['Family'].value_counts().idxmax()
    genus_majority = cluster_no['Genus'].value_counts().idxmax()
    species_majority = cluster_no['Species'].value_counts().idxmax()
    majority.append(family_majority)
    majority.append(genus_majority)
    majority.append(species_majority)

majority_df = pd.DataFrame(columns =[['Family', 'Genus','Species']] )
majority_labels = [majority[i:i + 3] for i in range(0, len(majority), 3)] 
for row in majority_labels:
    majority_df.loc[len(majority_df)] = row

display(majority_df)

,Family,Genus,Species
0,Hylidae,Hypsiboas,HypsiboasCinerascens
1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2,Hylidae,Hypsiboas,HypsiboasCordobae
3,Dendrobatidae,Ameerega,Ameeregatrivittata
4,Leptodactylidae,Adenomera,AdenomeraHylaedactylus


### (c) Calculate the average Hamming distance, Hamming score, and Hamming loss

In [20]:
h_score = []
h_distance = []
h_loss = []
for n in range(best_k):
    cluster_no = df_labels[df_labels['labels'] == n]
    for col in ['Family', 'Genus', 'Species']:
        majority_value =  majority_df[col].loc[n]
        majority_value = majority_value.iloc[0]
        
        # I used chatgpt to help create the below as I was researching how to calculate hamming distance, hamming score, and hamming loss in python with chatgpt
        miss = sum(cluster_no[col] != majority_value) # hamming distance
        accurate = sum(cluster_no[col] == majority_value) #hamming score is accurate / total

        hamming_score = accurate/len(df_labels)
        h_score.append(hamming_score)

        hamming_distance = miss
        h_distance.append(hamming_distance)

        hamming_loss = 1-hamming_score
        h_loss.append(hamming_loss)
        
print(f'Hamming Score: {np.average(h_score)}')
print(f'Hamming Distance: {np.average(h_distance)}')
print(f'Hamming Loss: {np.average(h_loss)}')





    


Hamming Score: 0.1554042158906648
Hamming Distance: 320.8666666666667
Hamming Loss: 0.8445957841093351


In [21]:
# combining all the above to iterate 50 times
# I did use chat gpt to debug some parts of the below code to see why my iterations were not properly outputting results
final_hscore = []
final_hdist= []
final_hloss = []

for i in range(1,51):

    print(f'iteration:{i}')
    
    sil = []
    range_n_clusters = [i for i in range(2,51)]

    for n_clusters in range_n_clusters:
        clusterer = KMeans(n_clusters= n_clusters, n_init= 'auto')
        cluster_labels = clusterer.fit_predict(df_features)
        sil_avg = silhouette_score(df_features, cluster_labels)
        sil.append(sil_avg)
        
    best_k2 = sil.index(max(sil)) + 2
    print(f'value of best k in this iteration is {best_k2}')

    kmeans2 = KMeans(n_clusters= best_k2, n_init= 'auto')
    kmeans2.fit(df_features)
    labels2 = kmeans2.labels_

    df_labels['labels'] = labels2
    maj = []

    for n in range(best_k2):
        cluster_no2 = df_labels[df_labels['labels'] == n]

        if cluster_no2.empty:
            print(f' cluster {n} is empty')
            continue

        family_majority2 = cluster_no2['Family'].value_counts().idxmax()
        genus_majority2 = cluster_no2['Genus'].value_counts().idxmax()
        species_majority2 = cluster_no2['Species'].value_counts().idxmax()
        maj.append(family_majority2)
        maj.append(genus_majority2)
        maj.append(species_majority2)

    
    maj_df = pd.DataFrame(columns =[['Family', 'Genus','Species']] )
    maj_labels = [maj[i:i + 3] for i in range(0, len(maj), 3)]

    for row in maj_labels:
        maj_df.loc[len(maj_df)] = row

    display(maj_df)
    
    h_score2 = []
    h_distance2 = []
    h_loss2 = []

    for n in range(best_k2):
        cluster_no2 = df_labels[df_labels['labels'] == n]
        for col in ['Family', 'Genus', 'Species']:
            
            if n not in maj_df.index:
                print(f'index {n} not found in maj_df')
                continue

            if col not in maj_df.columns:
                print(f'{col} not found')
                continue
            
            maj_value =  maj_df[col].loc[n]
            maj_value = maj_value.iloc[0]
            
            miss2 = sum(cluster_no2[col] != maj_value) 
            accurate2 = sum(cluster_no2[col] == maj_value)

            hamming_score2 = accurate2/len(df_labels)
            h_score2.append(hamming_score2)

            hamming_distance2 = miss2
            h_distance2.append(hamming_distance2)

            hamming_loss2 = 1-hamming_score2
            h_loss2.append(hamming_loss2)

    print(f'Hamming Score: {np.average(h_score2)}')
    print(f'Hamming Distance: {np.average(h_distance2)}')
    print(f'Hamming Loss: {np.average(h_loss2)}')

    avg_hscore = np.average(h_score2)
    avg_hdist = np.average(h_distance2)
    avg_hloss = np.average(h_loss2)

    final_hscore.append(avg_hscore)
    final_hdist.append(avg_hdist)
    final_hloss.append(avg_hloss)

print(f'Final Average Hamming score: {np.average(final_hscore)}')
print(f'Final Average Hamming distance: {np.average(final_hdist)}')
print(f'Final Average Hamming loss: {np.average(final_hloss)}')

        

iteration:1
value of best k in this iteration is 3


,Family,Genus,Species
0,Hylidae,Hypsiboas,HypsiboasCordobae
1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2,Hylidae,Hypsiboas,HypsiboasCordobae


Hamming Score: 0.23440660952822176
Hamming Distance: 711.7777777777778
Hamming Loss: 0.7655933904717782
iteration:2
value of best k in this iteration is 7


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Hylidae,Hypsiboas,HypsiboasCinerascens
3,Hylidae,Leptodactylus,LeptodactylusFuscus
4,Leptodactylidae,Adenomera,AdenomeraAndre
5,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
6,Leptodactylidae,Adenomera,AdenomeraAndre


Hamming Score: 0.115927065753334
Hamming Distance: 193.76190476190476
Hamming Loss: 0.8840729342466658
iteration:3
value of best k in this iteration is 4


...

Hamming Score: 0.19439425526986334
Hamming Distance: 400.0833333333333
Hamming Loss: 0.8056057447301366
iteration:49
value of best k in this iteration is 4


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Hylidae,Hypsiboas,HypsiboasCinerascens
3,Dendrobatidae,Ameerega,Ameeregatrivittata


Hamming Score: 0.19442900162149643
Hamming Distance: 399.8333333333333
Hamming Loss: 0.8055709983785034
iteration:50
value of best k in this iteration is 4


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Hylidae,Hypsiboas,HypsiboasCinerascens
3,Dendrobatidae,Ameerega,Ameeregatrivittata


Hamming Score: 0.18859161454713916
Hamming Distance: 441.8333333333333
Hamming Loss: 0.8114083854528608
Final Average Hamming score: 0.18306427523522728
Final Average Hamming distance: 420.6163492063492
Final Average Hamming loss: 0.8169357247647727


## 3. ISLR 12.6.2

ISLR 12.6.2 is attached in a separate pdf